In [1]:
import sys
sys.dont_write_bytecode = True
from collections import OrderedDict
import multiprocessing
import math
from breeder_blanket_model_maker import *
from breeder_blanket_model_maker.rewrite_cad_files import *
from breeder_blanket_model_maker.serpent_tools import *
from breeder_blanket_model_maker.make_serpent_input_file import *

from neutronics_material_maker.nmm import *
from neutronics_material_maker.examples import *


def define_blanket_geometry_parmeters(blanket_type,input_files,output_directory,poloidal_lithium_lead_in_mm = 35.4):

    poloidal_lithium_lead_in_mm = float(poloidal_lithium_lead_in_mm)
    smd = 274. * 1000000.  # units Pa

    cooling_plates_channel_poloidal_mm = 3.0
    cooling_plates_channel_radial_mm = 3.0
    cooling_plates_channel_toroidal_mm = 6.0
    first_wall_channel_radial_mm = 10.0
    first_wall_channel_poloidal_mm = 15.0
    first_wall_channel_pitch_mm = 5.2

    pressure_coolant = 10. * 1000000.  # in units Pa

    first_walls_thickness = (first_wall_channel_radial_mm/1000) + math.sqrt((pressure_coolant * (poloidal_lithium_lead_in_mm/1000) * (poloidal_lithium_lead_in_mm/1000) ) / (4. * smd))  # units in m
    first_walls_thickness_mm = first_walls_thickness *1000

    poloidal_cooling_plate_mm = (pressure_coolant * (poloidal_lithium_lead_in_mm)) / (1.1 * smd)

    poloidal_cooling_plate_mm = max(poloidal_cooling_plate_mm, 0.002*1000) + cooling_plates_channel_poloidal_mm

    cooling_channel_offset_from_first_wall = (first_walls_thickness_mm - first_wall_channel_radial_mm) / 2.0

    if cooling_channel_offset_from_first_wall < 1.0:
        raise ValueError('first wall is too thin at ',cooling_channel_offset_from_first_wall, 'mm')

    poloidal_segmentations_ordered_dict=OrderedDict()
    poloidal_segmentations_ordered_dict['lithium_lead']=poloidal_lithium_lead_in_mm
    poloidal_segmentations_ordered_dict['cooling_plate_homogenised']=poloidal_cooling_plate_mm

    back_plates_thicknesses_ordered_dict=OrderedDict()  #these are special ordereddict types required in python 2.7 but in python 3.6 onwards dictionaries are ordered by default, sadly freecad is not yet avaialbe in python 3
    back_plates_thicknesses_ordered_dict['back_plate_1']=30
    back_plates_thicknesses_ordered_dict['back_helium']=50
    back_plates_thicknesses_ordered_dict['back_plate_2']=30
    back_plates_thicknesses_ordered_dict['back_lithium_lead']=25
    back_plates_thicknesses_ordered_dict['back_plate_3']=10

    first_wall_channel_poloidal_segmentations_dict=OrderedDict()
    first_wall_channel_poloidal_segmentations_dict['first_wall_material']=first_wall_channel_poloidal_mm
    first_wall_channel_poloidal_segmentations_dict['first_wall_coolant']=first_wall_channel_pitch_mm


    list_of_compressed_arguments=[]
    for module in input_files:

        blanket_geometry_parameters =  {

            'blanket_type' : blanket_type,
            'plasma_filename' :'/home/jshim/Eurofusion_baseline_2016/envelopes/plasma.step',
            'envelope_filename' : module,
            'output_folder' : output_directory,
            'output_files':['step','stl'],
            'first_wall_poloidal_fillet_radius' : 50,
            'armour_thickness' : 2,
            'first_wall_thickness' : first_walls_thickness_mm,
            'end_cap_thickness' : 25,
            'back_plates_thicknesses' : back_plates_thicknesses_ordered_dict,
            'poloidal_segmentations' : poloidal_segmentations_ordered_dict,

        }

        if module == input_files[13]:
            blanket_geometry_parameters.update({
                #The following three arguments are optional, if included the model will include cooling channels on the first wall
                'cooling_channel_offset_from_first_wall': cooling_channel_offset_from_first_wall,
                'first_wall_channel_radial_mm': first_wall_channel_radial_mm,
                'first_wall_channel_poloidal_segmentations': first_wall_channel_poloidal_segmentations_dict,  # 13.5,4.5

                # #The following two arguments are optional, if included the model will include a slice of blanket and cooling channels in the cooling plate
                'cooling_plates_channel_poloidal_mm':cooling_plates_channel_poloidal_mm,
                'cooling_plates_channel_radial_mm':cooling_plates_channel_radial_mm,

                'output_files':['step','stl','umesh'],
                })



        list_of_compressed_arguments.append(blanket_geometry_parameters)

    return list_of_compressed_arguments

def define_neutronics_materials(enrichment_fraction):

    mat_lithium_lead =Compound('Pb84.2Li15.8',
                              density_atoms_per_barn_per_cm=3.2720171E-2,
                              enriched_isotopes=[Isotope('Li',6,abundance=enrichment_fraction),
                                                 Isotope('Li',7,abundance=1.0-enrichment_fraction)])

    material_dictionary = {  'armour': mat_Tungsten,
                             'lithium_lead': mat_lithium_lead,
                             'back_plate_1': mat_Eurofer,
                             'back_plate_2': mat_Eurofer,
                             'back_plate_3': mat_Eurofer,
                             'back_lithium_lead': mat_lithium_lead,
                             'back_helium': mat_He_coolant_back_plate,
                             'cooling_plate_homogenised': mat_cooling_plates_homogenised,
                             'end_caps_homogenised': mat_end_caps_homogenised,
                             'first_wall_homogenised': mat_first_wall_homogenised,
                             'plasma': mat_DT_plasma,
                             'central_solenoid': mat_central_solenoid_m25,
                             'divertor_1st_layer': mat_divertor_layer_1_m15, 
                             'divertor_2nd_layer': mat_divertor_layer_2_m74, 
                             'divertor_3rd_layer': mat_divertor_layer_3_m15,
                             'divertor_4th_layer': mat_divertor_layer_4_m75,
                             'manifolder': mat_VV_Body_m60 ,
                             'ports': mat_TF_Casing_m50,
                             #'shell': , # outer shell ignored
                             'shield': mat_TF_Casing_m50,
                             'tf_case': mat_TF_Casing_m50,
                             'tf_coils': mat_TF_Magnet_m25,
                             'vacuum_1st_layer': mat_VV_Shell_m50,
                             'vacuum_2nd_layer': mat_VV_Body_m60,
                             'vacuum_3rd_layer': mat_VV_Shell_m50,
                             'vaccum_vessel_shield': mat_ShieldPort_m60,
                             'blanket_support':mat_Eurofer,

                             'first_wall_coolant':mat_He_in_first_walls,
                             'first_wall_material':mat_He_in_first_walls,

                             'slice_lithium_lead':mat_lithium_lead,
                             'slice_armour':mat_Tungsten,
                             'slice_first_wall_material':mat_Eurofer,
                             'slice_first_wall_homogenised':mat_first_wall_homogenised, #ignore
                             'slice_first_wall_coolant':mat_He_in_first_walls,
                             'slice_cooling_plate_material':mat_cooling_plates_homogenised,
                             'slice_cooling_plate_coolant':mat_He_in_coolant_plates,
                             'slice_back_plate_1':mat_Eurofer,
                             'slice_back_helium':mat_He_coolant_back_plate,
                             'slice_back_plate_2':mat_Eurofer,
                             'slice_back_lithium_lead':mat_lithium_lead,
                             'slice_back_plate_3':mat_Eurofer,
                          }
    return material_dictionary

def define_neutronics_model_parmeters(list_detailed_modules_parts,material_dictionary,output_directory,nps=1e7,**kwargs):

    neutronics_parameters= { 'output_folder':output_directory,
                             'parts':list_detailed_modules_parts,
                             'include_umesh':True,
                             'output_folder_stl':os.path.join(output_directory,'stl'),
                             'material_dictionary':material_dictionary,
                             'plot_serpent_geometry':True,
                             'nps':nps,
                             'tallies':[{'name': 'tbr',
                                         'bodies': ['lithium_lead'],
                                         'mt_number':-55,
                                         'particle_type':'n'},
                                        {'name': 'neutron_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-4,
                                         'particle_type':'n'},
                                        {'name': 'photon_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-26,
                                         'particle_type':'p'},
                                       ]
                             }

    return neutronics_parameters


output_directory='/home/jshim/detailed_HCLL'
list_of_geometry_parameters = define_blanket_geometry_parmeters(blanket_type ='HCLL',
                                                                input_files= ['/home/jshim/Eurofusion_baseline_2016/envelopes/mod' + str(x) + '.step' for x in range(1,27)],#27,
                                                                output_directory = output_directory,
                                                                poloidal_lithium_lead_in_mm =80.0)


example materials imported


In [2]:
list_of_detailed_modules_parts = detailed_module(list_of_geometry_parameters)



Creating detailed HCLL
'read_in_envelope_file'  5.25 ms
'read_in_envelope_file'  10.26 ms
'read_in_envelope_file'  10.07 ms
'read_in_envelope_file'  9.62 ms
'read_in_envelope_file'  9.80 ms
'read_in_envelope_file'  9.82 ms
'read_in_envelope_file'  9.69 ms
'find_envelope_back_face'  509.41 ms
'find_envelope_front_face'  10.39 ms
'find_front_face_midpoint'  0.23 ms
'envelope_front_face_id'  10.41 ms
'find_envelope_back_face'  660.28 ms
'find_envelope_front_face'  9.87 ms
'find_front_face_midpoint'  0.24 ms
'envelope_front_face_id'  9.75 ms
'find_envelope_back_face_id'  566.15 ms
'find_front_face_polodial_edges_to_fillet'  0.09 ms
'find_front_face_torodial_edges_to_fillet'  0.08 ms
'filleted_envelope'  12.13 ms
'find_envelope_back_face'  1479.49 ms
'find_envelope_back_face'  1489.87 ms
'find_envelope_front_face'  10.52 ms
'find_front_face_midpoint'  0.24 ms
'find_envelope_front_face'  11.42 ms
'find_front_face_midpoint'  0.23 ms
'envelope_front_face_id'  10.01 ms
'envelope_front_face_id' 

'exstrude_and_cut_solids'  813.48 ms
'chop_up_poloidally'  1615.38 ms
'make_cylinder_slice'  0.18 ms
'find_envelope_back_face'  1376.85 ms
'find_envelope_front_face'  11.26 ms
'envelope_front_face_id'  13.94 ms
'find_end_cap_faces'  0.13 ms
'find_envelope_back_face'  3160.46 ms
'make_cylinder_slice'  2.06 ms
'find_envelope_front_face'  11.86 ms
'find_front_face_midpoint'  0.19 ms
'make_cylinder_slice'  0.28 ms
'make_cylinder_slice'  0.26 ms
'make_cylinder_slice'  0.25 ms
'chop_off_first_wall_armour'  234.73 ms
'make_cylinder_slice'  0.27 ms
'chop_of_end_caps'  152.72 ms
'make_cylinder_slice'  2.37 ms
'make_cylinder_slice'  0.21 ms
'make_cylinder_slice'  0.25 ms
'make_cylinder_slice'  0.26 ms
'save_components_as_step'  729.59 ms
'find_envelope_back_face'  2563.73 ms
'chop_off_back_plates'  482.46 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'find_envelope_back_face'  2970.00 ms
'find_envelope_front_face'  11.35 ms
'find_front_face_midpoint'  0.22 ms
'chop_of_end_caps'  163.8

'chop_off_first_wall'  103.60 ms
'read_in_envelope_file'  9.86 ms
'find_envelope_back_face'  1972.92 ms
'find_envelope_front_face'  13.68 ms
'find_end_cap_faces'  0.13 ms
'read_in_envelope_file'  11.64 ms
'find_envelope_back_face'  1458.94 ms
'find_envelope_front_face'  11.20 ms
'find_front_face_midpoint'  0.20 ms
'envelope_front_face_id'  11.13 ms
'exstrude_and_cut_solids'  2422.81 ms
'chop_up_poloidally'  5273.24 ms
'make_cylinder_slice'  0.16 ms
'find_envelope_back_face'  2097.51 ms
'find_envelope_front_face'  13.32 ms
'find_end_cap_faces'  0.14 ms
'find_envelope_back_face'  1373.45 ms
'find_envelope_front_face'  11.40 ms
'find_front_face_midpoint'  0.23 ms
'envelope_front_face_id'  14.12 ms
'find_envelope_back_face'  1944.00 ms
'find_envelope_front_face'  15.07 ms
'find_front_face_midpoint'  0.25 ms
'make_cylinder_slice'  2.03 ms
'make_cylinder_slice'  0.32 ms
'make_cylinder_slice'  0.42 ms
'find_envelope_back_face'  1573.25 ms
'find_envelope_front_face'  11.50 ms
'find_front_face_

'chop_off_first_wall_armour'  305.50 ms
'envelope_front_face_id'  10.21 ms
'find_envelope_back_face_id'  257.38 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.06 ms
'filleted_envelope'  11.14 ms
'make_cylinder_slice'  2.18 ms
'make_cylinder_slice'  0.31 ms
'make_cylinder_slice'  0.30 ms
'chop_off_back_plates'  734.37 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'make_cylinder_slice'  0.42 ms
'exstrude_and_cut_solids'  2063.62 ms
'make_cylinder_slice'  0.32 ms
'find_envelope_back_face'  759.82 ms
'find_envelope_front_face'  18.37 ms
'envelope_front_face_id'  20.63 ms
'find_end_cap_faces'  0.12 ms
'find_envelope_back_face'  590.97 ms
'find_envelope_front_face'  18.54 ms
'envelope_front_face_id'  18.00 ms
'find_end_cap_faces'  0.12 ms
'chop_off_first_wall_armour'  297.36 ms
'find_envelope_back_face'  1052.59 ms
'find_envelope_front_face'  8.42 ms
'find_end_cap_faces'  0.08 ms
'chop_off_first_wall_armour'  296.97 ms
'find_en

'filleted_envelope'  9.05 ms
'read_in_envelope_file'  8.96 ms
'find_envelope_back_face'  1156.63 ms
'find_envelope_front_face'  11.52 ms
'find_end_cap_faces'  0.12 ms
'find_envelope_back_face'  909.51 ms
'find_envelope_front_face'  16.13 ms
'envelope_front_face_id'  16.80 ms
'find_end_cap_faces'  0.13 ms
'find_envelope_back_face'  753.65 ms
'find_envelope_front_face'  15.20 ms
'envelope_front_face_id'  15.82 ms
'find_end_cap_faces'  0.12 ms
'find_envelope_back_face'  650.52 ms
'find_envelope_front_face'  9.79 ms
'find_front_face_midpoint'  0.20 ms
'envelope_front_face_id'  9.87 ms
'chop_off_first_wall_armour'  261.41 ms
'chop_off_first_wall_armour'  258.90 ms
'find_envelope_back_face'  980.10 ms
'find_envelope_front_face'  8.30 ms
'chop_off_first_wall'  66.52 ms
'find_envelope_back_face_id'  636.23 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.07 ms
'filleted_envelope'  11.73 ms
'exstrude_and_cut_solids'  1959.36 ms
'find_envelope_

('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
'add_cooling_pipes_to_div'  3610.17 ms
'make_cylinder_slice'  0.21 ms
'make_cylinder_slice'  0.56 ms
'make_cylinder_slice'  0.17 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.42 ms
'make_cylinder_slice'  0.67 ms
'make_cylinder_slice'  7.30 ms
'make_cylinder_slice'  0.11 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.22 ms
'make_cylinder_slice'  0.24 ms
'make_cylinder_slice'  0.24 ms
'save_components_

In [11]:
reload(breeder_blanket_model_maker)
reload(breeder_blanket_model_maker.rewrite_cad_files)
reload(breeder_blanket_model_maker.serpent_tools)
reload(breeder_blanket_model_maker.make_serpent_input_file)

<module 'breeder_blanket_model_maker.make_serpent_input_file' from '/usr/local/lib/python2.7/dist-packages/breeder_blanket_model_maker-0.3-py2.7.egg/breeder_blanket_model_maker/make_serpent_input_file.pyc'>

In [4]:
extra_parts = read_in_step_files_and_save_as_seperate_stl_files(read_folder='/home/jshim/Eurofusion_baseline_2016/reactor_step_files',
                                                                write_folder=os.path.join(output_directory,'stl'),
                                                                ignore_files=['shell.stp'])

('error making folder', '/home/jshim/detailed_HCLL/stl')
('read_folder', '/home/jshim/Eurofusion_baseline_2016/reactor_step_files')
['shield.stp', 'vacuum_1st_layer.stp', 'divertor_4th_layer.stp', 'divertor_3rd_layer.stp', 'divertor_2nd_layer.stp', 'vaccum_vessel_shield.stp', 'plasma.stp', 'divertor_1st_layer.stp', 'blanket_support.stp', 'central_solenoid.stp', 'vacuum_2nd_layer.stp', 'manifolder.stp', 'ports.stp', 'shell.stp', 'vacuum_3rd_layer.stp', 'tf_coils.stp', 'tf_case.stp']
shield.stp
vacuum_1st_layer.stp
divertor_4th_layer.stp
divertor_3rd_layer.stp
divertor_2nd_layer.stp
vaccum_vessel_shield.stp
plasma.stp
divertor_1st_layer.stp
blanket_support.stp
central_solenoid.stp
vacuum_2nd_layer.stp
manifolder.stp
ports.stp
vacuum_3rd_layer.stp
tf_coils.stp
tf_case.stp


In [10]:

list_of_detailed_modules_parts[0].update(extra_parts)

In [11]:
material_dictionary=define_neutronics_materials(enrichment_fraction=0.8)

In [5]:
list_of_detailed_modules_parts.update(extra_parts)

In [12]:
neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=list_of_detailed_modules_parts,
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=100000)


In [38]:
neutronics_parameters['parts']

def find_components(list_of_detailed_modules_components):
    #print('parts = ',list_of_detailed_modules_components)
    dictionary_of_components=collections.defaultdict(list)
    if type(list_of_detailed_modules_components) !='list':
        list_of_detailed_modules_components=[list_of_detailed_modules_components]
    for item in neutronics_parameters['parts']:
        if type(item)==list:
            for entry in item:
                for key,value in entry.iteritems():
                    dictionary_of_components[key].append(value)
        else:
            for key,value in item.iteritems():
                dictionary_of_components[key].append(value)
    return dictionary_of_components

find_components(neutronics_parameters['parts'])

defaultdict(list,
            {'armour': [{'part': [<Shape object at 0x7f62ac8145e0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod1_1.stl']},
              {'part': [<Shape object at 0x7f62ac144e90>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod2.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod2_1.stl']},
              {'part': [<Shape object at 0x7f62ac374930>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod3.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod3_1.stl']},
              {'part': [<Shape object at 0x7f62ac077300>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod4.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod4_1.stl']},
              {'part': [<Shape object at 0x7f62ac54cb60>],
     

In [35]:


    #print('parts = ',list_of_detailed_modules_components)
dictionary_of_components=collections.defaultdict(list)
#if type(list_of_detailed_modules_components) !='list':
#    list_of_detailed_modules_components=[list_of_detailed_modules_components]
for item in neutronics_parameters['parts']:
    if type(item)==list:
        for entry in item:
            for key,value in entry.iteritems():
                dictionary_of_components[key].append(value)
    else:
        for key,value in item.iteritems():
            dictionary_of_components[key].append(value)
dictionary_of_components

defaultdict(list,
            {'armour': [{'part': [<Shape object at 0x7f62ac8145e0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod1_1.stl']},
              {'part': [<Shape object at 0x7f62ac144e90>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod2.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod2_1.stl']},
              {'part': [<Shape object at 0x7f62ac374930>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod3.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod3_1.stl']},
              {'part': [<Shape object at 0x7f62ac077300>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod4.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod4_1.stl']},
              {'part': [<Shape object at 0x7f62ac54cb60>],
     

In [31]:
neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=[list_of_detailed_modules_parts],
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=100000)
directory_path_to_serpent_output, number_of_stl_parts = make_serpent_stl_based_input_file(neutronics_parameters)


AttributeError: 'list' object has no attribute 'iteritems'

In [31]:

def create_matfile_with_material_names(output_folder):
    #read in the matfile line by line, looks up the step files in the material dictionary
    fname = os.path.join(output_folder,'matfile')
    with open(fname) as f:
        content = f.readlines()


    new_content=[content[0]]
    for i, line in enumerate(content[1:]):
        filename=os.path.basename(line)
        for material_name in material_dictionary.keys():
            if filename.startswith(material_name):
                new_content.append(material_dictionary[material_name].material_card_name+'\n')
#                print('found', filename, material_name,material_dictionary[material_name].material_card_name )
                break
    
    fname = os.path.join(output_folder,'matfile_renamed')
    file_contents = ''.join(new_content)
    with open(fname, 'w') as f:   
        f.write(file_contents)
        

    
create_matfile_with_material_names(output_directory)

192063


In [8]:
neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=list_of_detailed_modules_parts,
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=10000)


In [9]:
directory_path_to_serpent_output, number_of_stl_parts = make_serpent_stl_based_input_file(neutronics_parameters)


/home/jshim/detailed_HCLL/stl
/home/jshim/detailed_HCLL
/home/jshim/detailed_HCLL
('relative director for stl files ', 'stl')
body slice_first_wall_homogenised-b slice_first_wall_homogenised-c 
body tf_case-b tf_case-c 
body back_plate_2-b back_plate_2-c 
body back_lithium_lead-b back_lithium_lead-c 
body shield-b shield-c 
body armour-b armour-c 
body tf_coils-b tf_coils-c 
body slice_back_plate_2-b slice_back_plate_2-c 
body slice_back_plate_3-b slice_back_plate_3-c 
body slice_back_plate_1-b slice_back_plate_1-c 
body divertor_2nd_layer-b divertor_2nd_layer-c 
body back_helium-b back_helium-c 
body first_wall_material-b first_wall_material-c 
body slice_back_helium-b slice_back_helium-c 
body vacuum_2nd_layer-b vacuum_2nd_layer-c 
body slice_first_wall_coolant-b slice_first_wall_coolant-c 
body cooling_plate_homogenised-b cooling_plate_homogenised-c 
body divertor_1st_layer-b divertor_1st_layer-c 
body plasma-b plasma-c 
body vacuum_1st_layer-b vacuum_1st_layer-c 
body first_wall_co

In [10]:
tally_dict = run_serpent_locally(directory_path_to_serpent_output)


/home/jshim/detailed_HCLL/serpent_input_file.serp
('File', '/home/jshim/detailed_HCLL/serpent_input_file.serp_det0.m', ' was not found.')
Was the serpent simulation run?
Has the _det0 file been output to the correct place?


SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
dictionary_of_parts = save_components_as_umesh(dictionary_of_parts = list_of_detailed_modules_parts, 
                                             output_folder = output_directory,
                                             mesh_component_prefix='slice_')

/home/jshim/detailed_HCLL/McCadConfig.txt
InitSurfNb               2100 
InitCellNb               100 
WriteCollisionFile       1 
MinimumInputSolidVolume  1.00 
MinimumVoidVolume        1.25e2 
MinimumSizeOfDecompositionFaceArea   50 
MinimumNumberOfSamplePoints 50 
MaximumNumberOfSamplePoints 100 
XResolution  0.001 
YResolution  0.001 
RResolution  0.00314 
MaxDecomposeDepth 15 
MaxCellExpressionLength 500 
Tolerance  1e-4 
VoidGenerate 0 
. 

'mccad_make_config_file'  0.35 ms
McCad -d slice_envelope_mod1.step -1
McCad -m D_slice_envelope_mod1.step
'mccad_convert_cad_to_csg'  174.80 ms
trelis /usr/local/lib/python2.7/dist-packages/breeder_blanket_model_maker-0.3-py2.7.egg/breeder_blanket_model_maker/mesh_with_trelis_for_serpent.py "output_dir='/home/jshim/detailed_HCLL'" "element_type='tet'" "inputs='/home/jshim/detailed_HCLL/step/slice_back_plate_2_mod1.step,/home/jshim/detailed_HCLL/step/slice_back_plate_3_mod1.step,/home/jshim/detailed_HCLL/step/slice_back_plate_1_mod1.step,/home

In [55]:
for entry in neutronics_parameters['parts']:
    for part_entry in neutronics_parameters['parts'][entry]['part']:
        #neutronics_parameters['parts'][entry]['part']=''
        print(part_entry)

In [56]:
import pickle
fileObject = open('/home/jshim/detailed_HCLL/saved_neutroic_materials','wb') 

pickle.dump(neutronics_parameters,fileObject)

In [58]:
fileObject = open('/home/jshim/detailed_HCLL/saved_neutroic_materials','r') 
retried_neutronics_parameters = pickle.load(fileObject)
retried_neutronics_parameters

{'include_umesh': True,
 'material_dictionary': {'armour': <neutronics_material_maker.nmm.Material at 0x7f153e3ff0d0>,
  'back_helium': <neutronics_material_maker.nmm.Compound at 0x7f153e12cb90>,
  'back_lithium_lead': <neutronics_material_maker.nmm.Compound at 0x7f153e3f2e90>,
  'back_plate_1': <neutronics_material_maker.nmm.Material at 0x7f155c299110>,
  'back_plate_2': <neutronics_material_maker.nmm.Material at 0x7f155c299110>,
  'back_plate_3': <neutronics_material_maker.nmm.Material at 0x7f155c299110>,
  'blanket_support': <neutronics_material_maker.nmm.Material at 0x7f155c299110>,
  'central_solenoid': <neutronics_material_maker.nmm.Homogenised_mixture at 0x7f153e13dd50>,
  'cooling_plate_homogenised': <neutronics_material_maker.nmm.Homogenised_mixture at 0x7f153e454dd0>,
  'divertor_1st_layer': <neutronics_material_maker.nmm.Material at 0x7f153e3ff0d0>,
  'divertor_2nd_layer': <neutronics_material_maker.nmm.Homogenised_mixture at 0x7f153e454250>,
  'divertor_3rd_layer': <neutron

In [33]:

neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=list_of_detailed_modules_parts,
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=10000)


In [59]:

fileObject = open('/home/jshim/detailed_HCLL/saved_neutroic_materials','r') 
retried_neutronics_parameters = pickle.load(fileObject)

directory_path_to_serpent_output,number_of_stl_parts= make_serpent_stl_based_input_file(retried_neutronics_parameters)

/home/jshim/detailed_HCLL/stl
/home/jshim/detailed_HCLL
/home/jshim/detailed_HCLL
('relative director for stl files ', 'stl')
body slice_first_wall_homogenised-b slice_first_wall_homogenised-c 
body back_lithium_lead-b back_lithium_lead-c 
body armour-b armour-c 
body slice_back_plate_2-b slice_back_plate_2-c 
body slice_back_plate_3-b slice_back_plate_3-c 
body slice_back_plate_1-b slice_back_plate_1-c 
body back_helium-b back_helium-c 
body first_wall_material-b first_wall_material-c 
body slice_back_helium-b slice_back_helium-c 
body slice_first_wall_coolant-b slice_first_wall_coolant-c 
body cooling_plate_homogenised-b cooling_plate_homogenised-c 
body first_wall_coolant-b first_wall_coolant-c 
body first_wall_homogenised-b first_wall_homogenised-c 
body slice_first_wall_material-b slice_first_wall_material-c 
body slice_armour-b slice_armour-c 
body slice_back_lithium_lead-b slice_back_lithium_lead-c 
body back_plate_1-b back_plate_1-c 
body back_plate_3-b back_plate_3-c 
body bac

In [143]:
neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=list_of_detailed_modules_parts,
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=10000)


In [12]:
directory_path_to_serpent_output='/home/jshim/detailed_HCLL/serpent_input_file.serp'
print(os.path.dirname(directory_path_to_serpent_output))
print(os.path.basename(directory_path_to_serpent_output))

/home/jshim/detailed_HCLL
serpent_input_file.serp


In [146]:
tally_dict = run_serpent_locally(directory_path_to_serpent_output)

/home/jshim/detailed_HCLL/serpent_input_file.serp
('File', '/home/jshim/detailed_HCLL/serpent_input_file.serp_det0.m', ' was not found.')
Was the serpent simulation run?
Has the _det0 file been output to the correct place?


SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [40]:
def read_mccad_csg_mcnp_csg(list_of_detailed_modules_parts):
    
    
    input_filename='D_'+os.path.basename('/home/jshim/detailed_HCLL/step/slice_envelope_mod1.step').split('.')[0]+'_MCNP.txt'
    input_dirname =os.path.dirname('/home/jshim/detailed_HCLL/step/slice_envelope_mod1.step')

    input_filename_and_path = os.path.join(input_dirname,input_filename)
    
    print('input_filename_and_path',input_filename_and_path)

    with open(input_filename_and_path) as f:
        contents = f.readlines()
    #print(contents)

        
    for i, line in enumerate(contents):
        if line.strip() == '':
            break
        if len(line)>0:
            if line[0].isdigit():
                #print('cell found')
                #print(line)
                cell_line = 'cell 10    0   fill all_um_geometry ' +' '.join(line.split()[2:])
    print('new cell_line',cell_line)
    
    
    surface_translate_dict= {'P':'plane','PY':'py'}
    
    new_surfaces =[]
    for line in contents[i+1:]:
        if line.strip() == '':
            break
        if len(line)>0:
            if line[0].isdigit():
                #print('surface found')
                #print(line)
                chop_up = line.split()
                new_surfaces.append(' '.join([chop_up[0],surface_translate_dict[chop_up[1]],' '.join(chop_up[2:]) ]))
                
    return cell_line, new_surfaces
        
read_mccad_csg_mcnp_csg('/home/jshim/detailed_HCLL/step/slice_envelope_mod1.step')        

('input_filename_and_path', '/home/jshim/detailed_HCLL/step/D_slice_envelope_mod1_MCNP.txt')
('new cell_line', 'cell 10    0   fill all_um_geometry (-2102 2104 2100 -2105 2101 2103)')


('cell 10    0   fill all_um_geometry (-2102 2104 2100 -2105 2101 2103)',
 ['2100 py 0.0000000',
  '2101 plane 0.1736482 -0.9848078 0.0000000 0.9999999',
  '2102 plane 0.4662963 0.0407957 -0.8836874 710.0643055',
  '2103 plane 0.4662963 0.0407957 -0.8836874 691.1670925',
  '2104 plane 0.8803247 0.0770184 0.4680775 286.4303991',
  '2105 plane 0.8803247 0.0770184 0.4680775 341.3304003'])